In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import numpy as np
import json
!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
!conda install -c conda-forge folium=0.5.0 --yes
import folium
print('Libraries imported.')

ModuleNotFoundError: No module named 'bs4'

In [2]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(source, 'lxml')

FSA = []
for post in soup.find_all('td'):
    code = post.text
    if code == '':
        break
    else:
        last_chr = int(len(code))
        last_chr = last_chr - 1
        if code[last_chr] == '\n':
            code = code[:-1]
        FSA.append(code)
post_dict = {}
postcode = []
burrough = []
neighbourhood = []
while len(FSA) > 2:
    postcode.append(FSA[0])
    burrough.append(FSA[1])
    neighbourhood.append(FSA[2])
    del FSA[0:3]

post_dic = {'Postal Code':postcode, 'Burrough':burrough, 'Neighborhood':neighbourhood}

df_post = pd.DataFrame(post_dic)

df_post = df_post[df_post.Burrough != 'Not assigned']

df_post = df_post.groupby(['Postal Code', 'Burrough'])['Neighborhood'].apply(', '.join).reset_index()

df_post.loc[df_post.Neighborhood == 'Not assigned', 'Neighborhood'] = df_post.Burrough

#print(df_post)

df_post.shape

NameError: name 'requests' is not defined

In [3]:
df_geo = pd.read_csv('https://cocl.us/Geospatial_data')
df_new = pd.merge(df_post, df_geo, on='Postal Code')
df_new.head()

NameError: name 'pd' is not defined

In [4]:
address = 'Toronto, CN'

geolocater = Nominatim()
location = geolocater.geocode(address)
latitude = location.longitude
longitude = location.longitude
print('The geographical coordinate of Toronto, Cananda are {}, {}'.format(latitude, longitude))

NameError: name 'Nominatim' is not defined

In [5]:
map_Toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

for lat,lng, burough, neighborhood in zip(df_new['Latitude'], df_new['Longitude'], df_new['Burrough'], df_new['Neighborhood']):
    label = '{}, {}'.format(neighborhood, burough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker([lat, lng],
                        radius=5,
                        popup=label,
                        color='blue',
                        fill=True,
                        fill_color='#3186cc',
                        fill_opacity=0.7,
                        parse_html=False).add_to(map_Toronto)
    
map_Toronto

NameError: name 'folium' is not defined

In [6]:
Toronto_west = df_new[df_new['Burrough'] == 'West Toronto'].reset_index(drop=True)
Toronto_east = df_new[df_new['Burrough'] == 'East Toronto'].reset_index(drop=True)
Toronto_down = df_new[df_new['Burrough'] == 'Downtown Toronto'].reset_index(drop=True)
Toronto_central = df_new[df_new['Burrough'] == 'Central Toronto'].reset_index(drop=True)
Toro = pd.concat([Toronto_west, Toronto_east,Toronto_down,Toronto_central])
Toro.reset_index(drop=True, inplace=True)
Toro

NameError: name 'df_new' is not defined

In [7]:
### TORONTO NEIGHBORHOODS -- WEST, EAST, DOWNTOWN and CENTRAL

map_Toronto_neigh = folium.Map(location=[latitude, longitude], zoom_start=10)

for lat,lng, burough, neighborhood in zip(Toro['Latitude'], Toro['Longitude'], Toro['Burrough'], Toro['Neighborhood']):
    label = '{}, {}'.format(neighborhood, burough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker([lat, lng],
                        radius=5,
                        popup=label,
                        color='blue',
                        fill=True,
                        fill_color='#3186cc',
                        fill_opacity=0.7,
                        parse_html=False).add_to(map_Toronto_neigh)
    
map_Toronto_neigh

NameError: name 'folium' is not defined

In [8]:
CLIENT_ID = '3M5MD4WTFDBCTGKO2CFVH120NXIJ1R43IRBJTVU0BGGIMSUP'
CLIENT_SECRET = 'AJRZ10JWEYOTWU44WM3JC04XJKQN2Y3XB2T5DHS5JFUMOYGX'
VERSION = '20180605'

In [9]:
radius = 500
LIMIT = 100
#url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
#     CLIENT_ID,
#     CLIENT_SECRET,
#     VERSION,
#     neighborhood_latitude,
#     neighborhood_longitude,
#     radius,
#     LIMIT)

In [10]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID,
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
        results = requests.get(url).json()['response']['groups'][0]['items']
        venues_list.append([(
            name, 
            lat,
            lng, 
            v['venue']['name'],
            v['venue']['location']['lat'],
            v['venue']['location']['lng'],
            v['venue']['categories'][0]['name']) for v in results])
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                            'Neighborhood Latitude', 
                            'Neighborhood Longitude', 
                            'Venue', 
                            'Venue Latitude', 
                            'Venue Longitude', 
                            'Venue Category']
    return(nearby_venues)

In [11]:

#print(Toro['Neighborhood'][1])
toro_venues = getNearbyVenues(names=Toro['Neighborhood'], latitudes=Toro['Latitude'], longitudes=Toro['Longitude'])

NameError: name 'Toro' is not defined

In [12]:
print(toro_venues.shape)
print(toro_venues.head())

NameError: name 'toro_venues' is not defined

In [13]:
toro_venues.groupby("Neighborhood").count()

NameError: name 'toro_venues' is not defined

In [14]:
print('There are {} unique categories.'.format(len(toro_venues['Venue Category'].unique())))

NameError: name 'toro_venues' is not defined

In [15]:
toro_onehot = pd.get_dummies(toro_venues[['Venue Category']], prefix="", prefix_sep="")
toro_onehot['Neighborhood'] = toro_venues['Neighborhood']
fixed_columns = [toro_onehot.columns[-1]] + list(toro_onehot.columns[:-1])
toro_onehot = toro_onehot[fixed_columns]
toro_onehot.head()
toro_onehot.shape

NameError: name 'pd' is not defined

In [16]:
toro_group = toro_onehot.groupby('Neighborhood').mean().reset_index()
toro_group.head()
toro_group.shape

NameError: name 'toro_onehot' is not defined

In [17]:
num_top_venues = 5
for hood in toro_group['Neighborhood']:
    print("----"+hood+"----")
    temp = toro_group[toro_group['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue', 'freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', 
ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

NameError: name 'toro_group' is not defined

In [18]:

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    return row_categories_sorted.index.values[0:num_top_venues]

In [19]:
num_top_venues = 10
indicators = ['st', 'nd', 'rd']
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))
        
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toro_group['Neighborhood']
for ind in np.arange(toro_group.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toro_group.iloc[ind, :], num_top_venues)
neighborhoods_venues_sorted

NameError: name 'np' is not defined

In [20]:
kclusters = 5
toro_group_clustering = toro_group.drop('Neighborhood', 1)
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toro_group_clustering)
kmeans.labels_[0:10]

NameError: name 'toro_group' is not defined

In [21]:
toro_merged = Toro
toro_merged['Cluster Labels'] = kmeans.labels_
toro_merged = toro_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
toro_merged.head()

NameError: name 'Toro' is not defined

In [22]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]
markers_colors = [0]
for lat, lon, poi, cluster in zip(toro_merged['Latitude'], toro_merged['Longitude'], toro_merged['Neighborhood'], toro_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
map_clusters

NameError: name 'folium' is not defined

In [23]:
toro_merged.loc[toro_merged['Cluster Labels'] == 0, toro_merged.columns[[1] + list(range(5, toro_merged.shape[1]))]]

NameError: name 'toro_merged' is not defined

In [24]:
toro_merged.loc[toro_merged['Cluster Labels'] == 1, toro_merged.columns[[1] + list(range(5, toro_merged.shape[1]))]]

NameError: name 'toro_merged' is not defined

In [25]:
toro_merged.loc[toro_merged['Cluster Labels'] == 2, toro_merged.columns[[1] + list(range(5, toro_merged.shape[1]))]]

NameError: name 'toro_merged' is not defined

In [26]:
toro_merged.loc[toro_merged['Cluster Labels'] == 3, toro_merged.columns[[1] + list(range(5, toro_merged.shape[1]))]]

NameError: name 'toro_merged' is not defined

In [ ]:
toro_merged.loc[toro_merged['Cluster Labels'] == 4, toro_merged.columns[[1] + list(range(5, toro_merged.shape[1]))]]